# Packages

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from requests import TooManyRedirects
import re
import omdb
import time

# API Set-up

In [2]:
api_key = "2857b99a" # Jimmy's key - limit 1000 calls a day
omdb.set_default('apikey', api_key)

# include tomatoes data by default
#omdb.set_default('tomatoes', True)

# Pulling Data from API

In [3]:
interstellar_df = pd.DataFrame(omdb.get(title='Interstellar', year=2014, fullplot=True, tomatoes=True, timeout=5))
interstellar_df.head(1)

,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,tomato_consensus,tomato_user_meter,tomato_user_rating,tomato_user_reviews,tomato_url,dvd,box_office,production,website,response
0,Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Jonathan Nolan, Christopher Nolan","Matthew McConaughey, Anne Hathaway, Jessica Ch...","Earth's future has been riddled by disasters, ...",...,N/A,N/A,N/A,N/A,https://www.rottentomatoes.com/m/interstellar_...,31 Mar 2015,"$188,020,017",N/A,N/A,True


In [4]:
interstellar_df['plot'][0]

"Earth's future has been riddled by disasters, famines, and droughts. There is only one way to ensure mankind's survival: Interstellar travel. A newly discovered wormhole in the far reaches of our solar system allows a team of astronauts to go where no man has gone before, a planet that may have the right environment to sustain human life."

In [5]:
interstellar_df['genre'][0]

'Adventure, Drama, Sci-Fi'

In [6]:
# set timeout of 5 seconds for this request
logan_df = pd.DataFrame(omdb.get(title='Logan', year=2017, fullplot=True, tomatoes=True, timeout=5))
logan_df.head(1)

,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,tomato_consensus,tomato_user_meter,tomato_user_rating,tomato_user_reviews,tomato_url,dvd,box_office,production,website,response
0,Logan,2017,R,03 Mar 2017,137 min,"Action, Drama, Sci-Fi",James Mangold,"James Mangold, Scott Frank, Michael Green","Hugh Jackman, Patrick Stewart, Dafne Keen",In 2029 the mutant population has shrunken sig...,...,N/A,N/A,N/A,N/A,https://www.rottentomatoes.com/m/logan_2017,20 Jun 2017,"$226,277,068",N/A,N/A,True


In [7]:
logan_df.iloc[0,:]['plot']

"In 2029 the mutant population has shrunken significantly due to genetically modified plants designed to reduce mutant powers and the X-Men have disbanded. Logan, whose power to self-heal is dwindling, has surrendered himself to alcohol and now earns a living as a chauffeur. He takes care of the ailing old Professor X whom he keeps hidden away. One day, a female stranger asks Logan to drive a girl named Laura to the Canadian border. At first he refuses, but the Professor has been waiting for a long time for her to appear. Laura possesses an extraordinary fighting prowess and is in many ways like Wolverine. She is pursued by sinister figures working for a powerful corporation; this is because they made her, with Logan's DNA. A decrepit Logan is forced to ask himself if he can or even wants to put his remaining powers to good use. It would appear that in the near-future, the times in which they were able put the world to rights with razor sharp claws and telepathic powers are now over."

In [8]:
logan_df['genre'][0]

'Action, Drama, Sci-Fi'

In [9]:
logan_df.columns

Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'ratings', 'metascore', 'imdb_rating', 'imdb_votes', 'imdb_id', 'type',
       'tomato_meter', 'tomato_image', 'tomato_rating', 'tomato_reviews',
       'tomato_fresh', 'tomato_rotten', 'tomato_consensus',
       'tomato_user_meter', 'tomato_user_rating', 'tomato_user_reviews',
       'tomato_url', 'dvd', 'box_office', 'production', 'website', 'response'],
      dtype='object')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=53ef1faa-b7a9-4637-b54c-3b067e58670d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>